In [3]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import os
from io import StringIO
import boto3
load_dotenv() 

host = os.getenv("HOST")
port = os.getenv("PORT")
database = os.getenv("DATABASE")
user = os.getenv("USER")
password = os.getenv("PASSWORD")
bucket = os.getenv('BUCKET')



In [6]:
path = '/home/pedrogarcia/Projetos/arquivos/videos-stats.csv'
file_name = 'youtube-video-stats.csv'
s3_client =boto3.client('s3')
try:
    s3_client.upload_file(path, bucket, file_name)
    print('Sucesso na inserção do arquivo no Bucket')
except Exception as e:
    print(f"An error occurred: {e}")

Sucesso na inserção do arquivo no Bucket
